# Harmony Regression 
This notebook provides condensed examples of using Harmony to perform specific tasks.  For more a general introduction and tutorial, see [Harmony API Introduction](https://harmony.earthdata.nasa.gov/notebook-example.html) or any of the example notebooks provided in the [Harmony repository](https://github.com/nasa/harmony). [Helper functions](./notebook_helpers) are used for making the calls found in this notebook.

## Prerequisites

 The dependencies for this notebook are listed in the [environment.yaml](./environment.yaml). To test or install locally, create the papermill environment used in the automated regression testing suite: 

`conda env create -f ./environment.yaml && conda activate papermill`

Also ensure you have a `.netrc` file located in the `test` directory of this repository.

## Set harmony_host_url and import libraries 

In [ ]:
#harmony_host_url = 'https://harmony.sit.earthdata.nasa.gov'
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'
#harmony_host_url = 'https://harmony.earthdata.nasa.gov'

# Import libraries used throughout the notebook
from notebook_helpers import get, post, show, get_data_urls, show_async, show_async_condensed, show_shape, print_async_status, check_bbox_subset, check_stac
import json
import intake

## Example Data

Harmony has produced example collections with artificial data but realistic structure to allow testing our services.  We have L3 and L2 NetCDF4 collections, and a shapefile collection.

In [ ]:
coverages_root = '{root}/{collection}/ogc-api-coverages/1.0.0/collections/{variable}/coverage/rangeset'

if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    l3_collection = 'C1234088182-EEDTEST'
    l3_zarr_collection = 'C1234088182-EEDTEST'
    l2_collection = 'C1233860183-EEDTEST'
    shapefile_collection = 'C1234530533-EEDTEST'
    l2ss_collection = 'C1234208438-POCLOUD'
    var_collection = 'C1234714698-EEDTEST'
elif harmony_host_url == 'https://harmony.earthdata.nasa.gov':
    l3_collection = 'C1756916832-XYZ_PROV'
    l3_zarr_collection = 'C1940468263-POCLOUD'
    l2_collection = 'C1756916832-XYZ_PROV'
    l2ss_collection = 'C1940473819-POCLOUD'

example_vars = ['red_var', 'green_var', 'blue_var', 'alpha_var']

### Sample Sync Request - Bounding Box and Temporal Subsetting with Reformatting

Harmony/gdal only working in UAT due to Harmony-600.

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    #By default, this reformats to tiff
    params = {
        'subset': [
            'lon(-20:90)', 
            'lat(0:60)', 
            'time("2020-01-15T00:00:00Z":"2020-01-15T01:00:00Z")']
    }
    response = get(
        coverages_root.format(
            root=harmony_host_url,
            collection=l3_collection, 
            variable='all'), 
        params=params)

    show(response, example_vars)

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    response = get(
    coverages_root.format(
        root=harmony_host_url,
        collection=l3_collection, 
        variable='green_var'), 
    params=params)

    show(response, color_index=1)

## Asynchronous Requests

Harmony/gdal only working in UAT due to Harmony-600.

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    response = get(
        coverages_root.format(
            root=harmony_host_url,
            collection=l3_collection, 
            variable='all'), 
        params={
            'subset': [
                'lon(-20:90)', 
                'lat(0:60)', 
                'time("2020-01-01T00:00:00Z":"2020-01-05T01:00:00Z")']})
    show_async_condensed(response)

## Cancel Async Requests

Harmony/gdal only working in UAT due to Harmony-600.

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    #Add 3 requests
    response1 = get(coverages_root.format(root=harmony_host_url, collection=l3_collection, variable='all'), params={'format': 'image/tiff', 'maxResults': '20'})
    response2 = get(coverages_root.format(root=harmony_host_url, collection=l3_collection, variable='all'), params={'format': 'image/tiff', 'maxResults': '20'})
    response3 = get(coverages_root.format(root=harmony_host_url, collection=l3_collection, variable='all'), params={'format': 'image/tiff', 'maxResults': '20'})

    #List the requests

    jobs_root = '{root}/jobs'
    my_jobs = jobs_root.format(root=harmony_host_url)
    response = get(my_jobs,params={'page': '1','limit': '10'})
    body = response.json()

    for job in body['jobs']:
        print_async_status(job)

    #Cancel one
    my_jobs_cancel_root=my_jobs+'/{job_id}/cancel'
    response = post(my_jobs_cancel_root.format(job_id=response3.json()['jobID']))

    print_async_status(response.json())

    assert response.json()['status'] == 'canceled'

# Basic Tests with Backend Services

### Shapefile Subsetting

Only available in UAT. Remove UAT or SIT if statements and test/modify requests when available in PROD.

#### PO.DAAC's Shapefile Subsetter 

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    show_shape('zip://./notebook_helpers/test_in-polygon.shp.zip')
    show(get('https://harmony.uat.earthdata.nasa.gov/service-results/harmony-uat-staging/public/shapefile_example/shapefile_r_001_249_20090109T000000.shp.zip'))

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    response = post(
        coverages_root.format(
            root=harmony_host_url,
            collection=shapefile_collection, 
            variable='all'), 
        data={ 'subset': 'time("2009-01-09T00:00:00Z":"2009-01-09T01:00:00Z")' },
        files={ 'shapefile': ('test_in-polygon.shp.zip', open('./notebook_helpers/test_in-polygon.shp.zip', 'rb'), 'application/shapefile+zip') }
    )

    try:
        show(response)
    except:
        print(response.text)


### Zarr Reformatter

In [ ]:
response = get(
    coverages_root.format(
        root=harmony_host_url,
        collection=l3_zarr_collection, 
        variable='all'), 
    params={
        'subset': 'time("2020-01-15T00:00:00Z":"2020-01-16T01:00:00Z")',
        'maxResults': '3'
        },
    headers = {'accept': 'application/x-zarr'})

zarr_response = show_async(response)

#### STAC outputs


In [ ]:
results = json.loads(zarr_response.content)
job = results['jobID']
print(job)

stac_root = '{root}/stac/{jobID}/{item}'

stac_cat = intake.open_stac_catalog(stac_root.format(root=harmony_host_url,jobID=job,item=''),name='Harmony output')
display(list(stac_cat))

In [ ]:
for i in range(len(list(stac_cat))):
    display(intake.open_stac_item(stac_root.format(root=harmony_host_url,jobID=job,item=i)))

entries = []
for id, entry in stac_cat.search('type').items():
    display(entry)
    entries.append(entry)

In [ ]:
# da = stac_cat[list(stac_cat)[0]][entries[0].describe()['name']].to_dask()
# da

In [ ]:
#da.plot.imshow()

### SWOT Reprojection Tool

Only available in UAT. Remove UAT or SIT if statements and test/modify requests when available in PROD.

In [ ]:
#Shows original test data for easy visual comparison

if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    response = get('https://harmony.uat.earthdata.nasa.gov/service-results/harmony-uat-staging/public/harmony_example_l2/nc/015_02_210_europe.nc')
    show(response, example_vars)

#### SWOT Reprojection Tool (Sync)

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    response = get(
        coverages_root.format(
            root=harmony_host_url,
            collection=l2_collection, 
            variable='all'), 
        params={
            'outputCrs': 'EPSG:4326',
            'subset': 'time("2020-01-15T16:00:00Z":"2020-01-15T17:00:00Z")'})
    show(response, example_vars)

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    response = get(
    coverages_root.format(
        root=harmony_host_url,
        collection=l2_collection, 
        variable='all'), 
    params={
        'outputCrs': '+proj=lcc +lat_1=43 +lat_2=62 +lat_0=30 +lon_0=10 +x_0=0 +y_0=0 +ellps=intl +units=m +no_defs',
        'interpolation': 'near',
        'scaleExtent': '-4000000,-1000000,5000000,7000000',
        'subset': 'time("2020-01-15T16:00:00Z":"2020-01-15T17:00:00Z")'})

    show(response, example_vars)

#### SWOT Reprojection Tool (Async)

Only available in UAT. Remove UAT or SIT if statements and test/modify requests when available in PROD.

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    response = get(
        coverages_root.format(
            root=harmony_host_url,
            collection=l2_collection, 
            variable='all'), 
        params={
            'outputCrs': '+proj=lcc +lat_1=43 +lat_2=62 +lat_0=30 +lon_0=10 +x_0=0 +y_0=0 +ellps=intl +units=m +no_defs',
            'interpolation': 'near',
            'scaleExtent': '-7000000,-1000000,8000000,8000000',
            'maxResults': '3'})

    show_async_condensed(response, example_vars)


### PODAAC L2 Subsetter

#### PODAAC L2 Subsetter (Sync)

In [ ]:
response = get(
    coverages_root.format(
        root=harmony_host_url,
        collection=l2ss_collection, 
        variable='all'), 
    params={
        'maxResults': '1',
        'subset': [
           'lon(-160:160)', 
           'lat(-80:80)',
           'time("2012-03-03T12:17:00":"2012-03-03T12:18:00")']
        })

if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    show(response, ['ssha'])
else:
    show(response, ['sea_surface_temperature'])

#### PODAAC L2 Subsetter (Async)

In [ ]:
response = get(
    coverages_root.format(
        root=harmony_host_url,
        collection=l2ss_collection, 
        variable='all'), 
    params={
        'maxResults': '3',
        'format': 'application/x-netcdf4',
        'subset': [
            'lon(-160:160)', 
            'lat(-80:80)',
             'time("2012-03-03T00:00:00Z":"2012-03-03T02:59:59Z")'
            ]})

if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    show_async_condensed(response, ['ssha'])
else:
    show_async_condensed(response, ['sea_surface_temperature'])

### SDS Variable Subsetter
Only available in UAT. Remove UAT or SIT if statements and test/modify requests when available in PROD.


#### SDS Variable Subsetter (Sync)

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    response = get(
        coverages_root.format(
            root=harmony_host_url,
            collection=var_collection, 
            variable='%2Fgt1l%2Fland_segments%2Fcanopy%2Fh_canopy'), 
        params={
            'granuleid':'G1238479514-EEDTEST'
        })

    show(response, ['/gt1l/land_segments/canopy/h_canopy'])

#### SDS Variable Subsetter (Async)

In [ ]:
if harmony_host_url == 'https://harmony.uat.earthdata.nasa.gov' or harmony_host_url == 'https://harmony.sit.earthdata.nasa.gov':
    response = get(
        coverages_root.format(
            root=harmony_host_url,
            collection=var_collection, 
            variable='%2Fgt1l%2Fland_segments%2Fcanopy%2Fh_canopy'), 
            params={'maxResults': '3'})

    show_async_condensed(response, ['/gt1l/land_segments/canopy/h_canopy'])

### ASF's GDAL Subsetter (These cells are conmented out until issues with the ASF service are resolved)

#### ASF's GDAL Subsetter (Sync)

In [ ]:
# response = get(
#     coverages_root.format(
#         root=harmony_host_url,
#         collection='C1225776654-ASF',
#         variable='science%2Fgrids%2Fdata%2Famplitude'), 
#         params={
#             'granuleId' : 'G1235282694-ASF',
#             'subset': [
#             'lon(37:40)', 
#             'lat(23:24)', 
#             'time("2014-10-30T15:00:00Z":"2014-10-30T15:59:00Z")']})
            
# show(response)


#### ASF's GDAL Subsetter (Async)

In [ ]:
# response = get(
#     coverages_root.format(
#         root=harmony_host_url,
#         collection='C1225776654-ASF',
#         variable='science%2Fgrids%2Fdata%2Famplitude'), 
#         params={
#             'subset': [
#             'lon(37:40)', 
#             'lat(23:24)', 
#             'time("2014-10-30T15:00:00Z":"2014-10-30T15:59:00Z")']})
            
# show_async_condensed(response)

_Minimizing time to science_